#### Thursday, May 9, 2024

<strike>mamba activate langchain3</strike>

mamba activate langchain4

This notebook was created from the code demonstrated from the Sam Witteveen video [Function Calling with Local Models & LangChain - Ollama, Llama3 & Phi-3](https://www.youtube.com/watch?v=Ss_GdU0KqE0)

This code is found in the repo [samwit/agent_tutorials](https://github.com/samwit/agent_tutorials/tree/main/ollama_agents/llama3_local) and is not in notebooks, but individual python files.

To run this notebook, I cloned langchain3 to langchain4, then on langchain4 ran ..

* pip uninstall langchain-experimental
* pip install langchain-experimental

This resulted in the following changes ...

    Installing collected packages: langchain-core, langchain-community, langchain, langchain-experimental
    Attempting uninstall: langchain-core
        Found existing installation: langchain-core 0.1.46
        Uninstalling langchain-core-0.1.46:
        Successfully uninstalled langchain-core-0.1.46
    Attempting uninstall: langchain-community
        Found existing installation: langchain-community 0.0.32
        Uninstalling langchain-community-0.0.32:
        Successfully uninstalled langchain-community-0.0.32
    Attempting uninstall: langchain
        Found existing installation: langchain 0.1.15
        Uninstalling langchain-0.1.15:
        Successfully uninstalled langchain-0.1.15
    Successfully installed langchain-0.1.19 langchain-community-0.0.38 langchain-core-0.1.52 langchain-experimental-0.0.58

(I did this because of weird errors I was getting when it ran in langchain3.)

We get one error with the phi3 model, but other than that, this notebook works. 


1) ollama_agents/llama3_local/testing_ollama.py

In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [2]:
# Local Llama3 
llm = ChatOllama(
    model="llama3",
    keep_alive=-1, # keep the model loaded indefinitely
    temperature=0,
    max_new_tokens=512)

In [3]:
prompt = ChatPromptTemplate.from_template("Write me a 500 word article on {topic} from the perspective of a {profession}. ")

In [4]:
# using LangChain Expressive Language chain syntax
chain = prompt | llm | StrOutputParser()

In [5]:
# This will generated the entire chain and then print the result
print(chain.invoke({"topic": "LLMs", "profession": "shipping magnate"}))

# 7.1s

**"LLMs: The Game-Changers for Shipping Magnates Like Me"**

As a shipping magnate, I've spent my fair share of time navigating the complexities of global trade and commerce. From managing fleets of vessels to negotiating with ports and customs officials, it's a never-ending battle to stay ahead of the curve. But in recent years, I've noticed a new player on the field that has the potential to revolutionize our industry: Large Language Models (LLMs).

At first, I was skeptical about these AI-powered language processing machines. What could they possibly do for me? But as I delved deeper into their capabilities, I realized that LLMs have the power to transform the way we operate in shipping.

First and foremost, LLMs can help us streamline our communication processes. Gone are the days of tedious email exchanges and phone calls with customs officials, ports, and other stakeholders. With an LLM-powered chatbot, we can automate these interactions, reducing errors and increasing efficiency

In [6]:
# Running it this way will stream the output to the console a token at a time.
for chunk in chain.stream({"topic": "LLMs", "profession": "shipping magnate"}):
    print(chunk, end="", flush=True)

# 11.6s

**"LLMs: The Game-Changers for Shipping Magnates Like Me"**

As a shipping magnate, I've spent my fair share of time navigating the complexities of global trade and commerce. From managing fleets of vessels to negotiating with ports and customs officials, there's no shortage of challenges in this industry. But one development that has caught my attention - and potentially changed the game for me and my competitors alike - is the rise of Large Language Models (LLMs).

At first glance, LLMs might seem like a distant cousin to the world of shipping. After all, they're AI-powered language processing systems designed to analyze and generate human-like text. But trust me, their impact on our industry has been nothing short of profound.

Let's start with the obvious: data analysis. As a shipping magnate, I need to stay on top of market trends, demand patterns, and supply chain disruptions. LLMs have revolutionized my ability to do just that. By processing vast amounts of text-based data - fro

2) ollama_agents/llama3_local/llama3_json.py

Output the result of the function call in JSON.

In [7]:
import json
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [8]:
json_schema = {
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
        "name": {"title": "Name", "description": "The person's name", "type": "string"},
        "age": {"title": "Age", "description": "The person's age", "type": "integer"},
        "favorite_food": {
            "title": "Fav Food",
            "description": "The person's favorite food",
            "type": "string",
        },
    },
    "required": ["name", "age","fav_food"],
}


In [9]:
llm = ChatOllama(
    model="llama3",
    format="json", # we need to set the output format to json
    keep_alive=-1, # keep the model loaded indefinitely
    temperature=0.1,
    max_new_tokens=512
    )

In [10]:
messages = [
    HumanMessage(
        content="Please tell me about a person using the following JSON schema:"
    ),
    HumanMessage(content="{schema}"),
    HumanMessage(
        content="Now, considering the schema, tell me about a person named John who is 35 years old and loves pizza."
    ),
]


In [11]:
prompt = ChatPromptTemplate.from_messages(messages)

In [12]:
#converting the json schema to a string
dumps = json.dumps(json_schema, indent=2)

In [13]:
# chain = prompt | llm | StrOutputParser()
chain = prompt | llm | JsonOutputParser()

In [14]:
response = chain.invoke({"schema": dumps})
print(response)
print(type(response))

# 3.1s

{'name': 'John', 'age': 35, 'hobbies': ['pizza']}
<class 'dict'>


Notice the different response we get if we change from a JsonOutputParser() to a StrOutputParser.

In [15]:
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"schema": dumps})
print(response)
print(type(response))

# notice all the white space following the {"name": "John", "age": 35, "hobbies": ["pizza"]} ...

{"name": "John", "age": 35, "hobbies": ["pizza"]}


<class 'str'>


So generally you want to stick with using the JsonOutputParser() rather than the StrOutputParser().

3) ollama_agents/llama3_local/llama3_ollama_structured_output.py

In [16]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.llms.ollama_functions import OllamaFunctions

In [17]:
# Pydantic Schema for structured response
class Person(BaseModel):
    name: str = Field(description="The person's name", required=True)
    height: float = Field(description="The person's height", required=True)
    hair_color: str = Field(description="The person's hair color")

In [18]:
context = """Alex is 5 feet tall. 
Claudia is 1 feet taller than Alex and jumps higher than him. 
Claudia is a brunette and Alex is blonde."""

In [19]:
# Prompt template llama3
prompt = PromptTemplate.from_template(
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a smart assistant take the following context and question below and return your answer in JSON.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
QUESTION: {question} \n
CONTEXT: {context} \n
JSON:
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
 """
)

In [20]:
# Chain
llm = OllamaFunctions(model="llama3", 
                      format="json", 
                      temperature=0)

In [21]:
structured_llm = llm.with_structured_output(Person)
chain = prompt | structured_llm

In [22]:
response = chain.invoke({
    "question": "Who is taller?",
    "context": context
    })

print(response)

# notice this answer is wrong!

name='Alex' height=5.0 hair_color='blonde'


In [23]:
response = chain.invoke({
    "question": "Who is taller, Claudia or Alex?",
    "context": context
    })

print(response)

name='' height=6.0 hair_color=''


4) ollama_agents/llama3_local/phi3_ollama_structured_output.py

In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.llms.ollama_functions import OllamaFunctions

# Schema for structured response
class Person(BaseModel):
    name: str = Field(description="The person's name", required=True)
    height: float = Field(description="The person's height", required=True)
    hair_color: str = Field(description="The person's hair color")

context = """Alex is 5 feet tall. 
Claudia is 1 feet taller than Alex and jumps higher than him. 
Claudia is a brunette and Alex is blonde."""


In [25]:
# Prompt template phi 3
prompt = PromptTemplate.from_template(
    """<|user|>{context}

QUESTION: {question}<|end|>
<|assistant|>AI: """
)


In [26]:
# Chain
llm = OllamaFunctions(model="phi3", 
                      format="json", 
                      temperature=0)

In [27]:
structured_llm = llm.with_structured_output(Person)
chain = prompt | structured_llm

In [28]:
response = chain.invoke({
    "question": "Who is taller?",
    "context": context
    })

print(response)

OutputParserException: Failed to parse Person from completion {"name": ["Alex", "Claudia"], "properties": {"name": {"values": [{"Alex": "string"}, {"Claudia": "string"}]}, "height": {"values": [5, 6]}, "hair_color": {"values": ["blonde", "brunette"]}}}. Got: 3 validation errors for Person
name
  str type expected (type=type_error.str)
height
  field required (type=value_error.missing)
hair_color
  field required (type=value_error.missing)

5) ollama_agents/llama3_local/llama3_ollama_functions.py

In [29]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.messages import HumanMessage

model = OllamaFunctions(
    model="llama3", 
    format="json"
    )


In [30]:
model = model.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

In [31]:
response = model.invoke("what is the weather in Singapore?")

print(response)

content='' additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "Singapore", "unit": "celsius"}'}} id='run-83b17237-4252-4c55-aab6-b0cd325938c5-0'


6) ollama_agents/llama3_local/phi3_ollama_functioncalling.py

In [32]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.messages import HumanMessage

model = OllamaFunctions(
    model="phi3", 
    keep_alive=-1,
    format="json"
    )

In [33]:
model = model.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

In [34]:
response = model.invoke("what is the weather in Singapore?")

print(response)

content='' additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "Singapore"}'}} id='run-6341c0ce-5ffb-42d5-a38e-750f6749f6f0-0'
